# Поддержка грядущего стандарта Ада 202х в GNAT Community Edition 2020

[![Binder](https://mybinder.org/badge_logo.svg)][mybinder]

В мае 2020г. вышла новая редакция компилятора Ada - [GNAT Community Edition 2020].
В ней, среди прочих изменений и улучшений, реализована значительная часть новых
возможностей [грядущего стандарта Ады] носящего условное название Ada 202x.

Давайте познакомимся с ними поближе. Благодаря [Jupyter Ada Kernel]
поэкспериментировать с примерами использования этих возможностей можно прямо
в бравзере. Для этого откройте данный "блокнот" просто перейдя по ссылке
на [mybinder].
(Предупреждение: загрузка блокнота может занять некоторое время необходимое для сборки и запуска контейнера).

Если Вы ранее не работали с блокнотами Jupyter - не пугайтесь, это не сложно.
Основная идея в том, что Вы может выполнять ячейки с кодом. Для этого нажмите
`Ctrl+Enter` либо воспользуйтесь меню или иконками. В [Jupyter Ada Kernel]
ячейка может содержать [спецификатор контекста], [оператор] или [определение],
либо несколько таких элементов, но одного типа.


## Активация Ada 202x

Первое, что нужно сделать, это указать компилятору, что Вы используете новый стандарт.
Обычно это делается при помощи ключа компилятора `-gnat2020`.
Указать его можно в коммандной строке или лучше в проектном файле.
Но в этом блокноте мы будем пользоваться директивой компилятора `pragma Ada_2020;`.
Следующая ячейка должна выполнятся первой, иначе последующие ячейки работать не будут!
(А ещё нам понадобится пакет `Ada.Text_IO;`.)

[GNAT Community Edition 2020]:
  https://blog.adacore.com/gnat-community-2020-is-here
  "GNAT Community 2020 is here!"

[грядущего стандарта Ады]:
  http://www.ada-auth.org/standards/ada2x.html
  "Ada 202x Language Reference Manual"

[Jupyter Ada Kernel]: https://github.com/reznikmm/jupyter

[mybinder]:
  https://mybinder.org/v2/gh/reznikmm/ada-howto/ce-2020?filepath=%2Fhome%2Fjovyan%2Fce2020.ipynb
  "Run me!"

[спецификатор контекста]:
  http://www.ada-auth.org/standards/rm12_w_tc1/html/RM-10-1-2.html#S0253
  "context clauses"

[оператор]:
  http://www.ada-auth.org/standards/rm12_w_tc1/html/RM-5-1.html#S0146
  "statements"

[определение]:
  http://www.ada-auth.org/standards/rm12_w_tc1/html/RM-3-11.html#S0087
  "declarative items"

In [1]:
pragma Ada_2020;
with Ada.Text_IO;

## Больше выразительности и удобства

Авторы стандарта стараются сделать язык более выразительным и удобным. К нововведениям этой группы можно отнести литералы пользовательских типов, агрегаты контейнеров, новые виды агрегатов массивов и записей и прочее. Давайте начнем с атрибута `'Image`.

### Атрибут `'Image` для всех типов

В новом стандарте атрибут `'Image` работает для всех типов, а не только для скалярных, как было раньше. (Если Вы пропустили изменения в стандарте с 
"[техническими правками 2016](http://www.ada-auth.org/standards/rm12_w_tc1/html/RM-3-5.html#I1681)",
то знайте, теперь префиксом атрибута может выступать не только тип, но и значение).

In [2]:
Integer_Array : array (1 .. 3) of Integer := (1, 2, 3);

In [3]:
Ada.Text_IO.Put_Line (Integer_Array'Image);


[ 1,  2,  3]


### Переопределение атрибута `'Image`

Также можно задать подпрограмму для отображения значений своего типа.
Причём авторы компилятора пошли другим путём, чем написано в стандарте.
(Видимо черновик стандарта будет обновлятся, если ARG приймет новый вариант этой функциональности).
Вариант от AdaCore для меня выглядит странно, да и то, что прописали в черновик - не идеал.

Основная идея в том, чтобы определить аспект `Put_Image` для вашего типа используя костыли из предопределённого пакета:

In [4]:
with Ada.Strings.Text_Output.Utils;

In [5]:
package Source_Locations is
   type Source_Location is record
      Line : Positive;
      Column : Positive;
   end record
     with Put_Image => My_Put_Image;

   procedure My_Put_Image
     (Sink  : in out Ada.Strings.Text_Output.Sink'Class;
      Value : Source_Location);
end Source_Locations;

package body Source_Locations is

   procedure My_Put_Image
     (Sink  : in out Ada.Strings.Text_Output.Sink'Class;
      Value : Source_Location)
   is
      Line   : constant String := Value.Line'Image;
      Column : constant String := Value.Column'Image;
      Result : constant String :=
        Line (2 .. Line'Last) & ':' & Column (2 .. Column'Last);
   begin
       Ada.Strings.Text_Output.Utils.Put_UTF_8 (Sink, Result);
   end My_Put_Image;

end Source_Locations;

In [6]:
Line_10 : Source_Locations.Source_Location := (Line => 10, Column => 1);

In [7]:
Ada.Text_IO.Put_Line ("Text position " & Line_10'Image);

Text position 10:1


### Числовые и строковые литералы для пользовательских типов 
    
Появилась возможность использовать литералы языка для своих типов.
Это позволяет избежать явного вызова функции преобразования.

Это удобно, когда вводишь свои строчные и числовые типы:

In [8]:
with Ada.Strings.Unbounded;
with Ada.Strings.UTF_Encoding.Wide_Wide_Strings;

In [9]:
type My_String is new Ada.Strings.Unbounded.Unbounded_String
  with String_Literal => From_String;

function From_String (Text : Wide_Wide_String) return My_String is
 (To_Unbounded_String
   (Ada.Strings.UTF_Encoding.Wide_Wide_Strings.Encode
     (Text)));

Five : My_String := "5";


In [10]:
Ada.Text_IO.Put_Line (To_String (Five));

5


### Итеративные ассоциации в агрегатах массивов

Это попытка скрестить итераторы `for` и агрегаты массивов:

In [11]:
type Integer_Matrix is array (1 .. 3, 1 .. 3) of Integer;

X_3x3 : Integer_Matrix :=
 (for K in 1 .. 3 =>
   (for J in 1 .. 3 => (K * 10 + J)));

In [12]:
Ada.Text_IO.Put_Line (X_3x3'Image);


[
 [ 11,  12,  13],

 [ 21,  22,  23],

 [ 31,  32,  33]]
